In [ ]:
import netCDF4 as nc
import scipy 
import os
import re
import xarray as xr
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import dask
from xmhw.xmhw import threshold, detect
from datetime import date
import statsmodels.api as sm
import statsmodels.formula.api as smf
import plotly.express as px
import plotly.graph_objects as go
import hashlib
import matplotlib.colors as mcolors

In [ ]:
input_location = 'Jurien' ## change this location ['Jurien', 'Jervis Bay', 'Maria Island']

In [ ]:
df_location = pd.read_csv('er_atrc_site.csv')
input_lat = df_location.latitude[df_location.location == input_location].mean()
input_lon = df_location.longitude[df_location.location == input_location].mean()

## Find mhws and temperatures in each location

In [ ]:
def summer_mhw_metric(input_location):
    infile = 'OBIS_kelp_sst.nc'    
    
    ds = xr.open_dataset(infile).sel(
        time = slice('1991-01-01','2024-12-31')).sel(
        lat = slice(input_lat-0.25, input_lat+0.25), lon = slice(input_lon-0.25, input_lon+0.25)).squeeze('zlev', drop=True).mean(dim=("lat", "lon"))
    ds_summer = ds.sel(time=ds.sst.time.dt.month.isin([12,1,2,3,4]))
    
    date = np.array(ds_summer.time,dtype='datetime64[ns]')
    df = pd.DataFrame(date, columns=['date']) 
    df['doy'] = df['date'].dt.dayofyear
    df['temp'] = pd.DataFrame(ds_summer.sst.values)
    t = ds.time.values
    sst = ds.sst.values
    clim = threshold(ds.sst, climatologyPeriod=[1991, 2020]) # threshold and climatology
    maxthresh = clim.seas.max().values 
    
    # Convert the xarray DataArray to a DataFrame
    climatology_df = clim.to_dataframe().reset_index()

    # Merge the two DataFrames on the 'doy' column
    merged_df = pd.merge(df, climatology_df, on='doy', how='left')  

    # add the dhd column, only keep positive intense values as our dhd
    merged_df['dhd50'] = (merged_df.temp - maxthresh).where((merged_df.temp - maxthresh)>0, 0)  
    merged_df['max_inten'] = merged_df.temp - maxthresh   ### intensity relative to different climatology values
    ## ==========================temp, cum_inten, max_inten ===========================================
    summer_mhw_metrix = merged_df[['temp', 'dhd50', 'max_inten']].set_index(merged_df.date)
    summer_mhw_metrix.columns = ['summer_temp', 'summer_dhd50', 'summer_inten']
    # get mean summer temp and sum of summer intense
    summer_mhw_metrix = summer_mhw_metrix.resample('AS-DEC').agg({'summer_temp': 'mean', 'summer_dhd50': 'sum', 'summer_inten': 'max'})
    # add one year
    summer_mhw_metrix['survey_year'] = summer_mhw_metrix.index.year+1
    
    ## ============================= dt dt =========================================================
    df = df.set_index('date')
    dt = 10
    dTdt = pd.DataFrame()
    for i in df.index.year.unique()[0:-1]:
        ssti = df[(df.index.year == i) & (df.index.month > 11) | (df.index.year == i+1) & (df.index.month <5)]
        
        # calculate n-day jumping mean
        jmeani = ssti.resample('10D').mean()

        # calculate dT as difference between consecutive 10-day means
        dT = jmeani.diff(periods=1)
        dTdt = pd.concat([dTdt, dT/dt])  #.mean()
        dTdt = dTdt.dropna()

    summer_dtdt = dTdt.resample('AS-DEC').max()
    summer_dtdt = summer_dtdt.rename(columns={'temp': 'summer_dtdt'}) 
    summer_dtdt['survey_year'] = summer_dtdt.index.year + 1
    
    summer_mhw_metrix = summer_mhw_metrix.merge(summer_dtdt, how='inner', on='survey_year')
    
    return summer_mhw_metrix

In [ ]:
# check the output metric df
summer_mhw_metrix = summer_mhw_metric(input_location).fillna(0)
summer_mhw_metrix #.head()

### kelp change all data

#### Site and year selection first

In [ ]:
def get_site_year(input_location):
    df_site_kelp = pd.read_csv("er_3pop_site.csv") 
    df_site_kelp = df_site_kelp[df_site_kelp.location == input_location]

    df_site_kelp = df_site_kelp.sort_values(by=['site_name', 'survey_year'])
    df_site_kelp['survey_datetime'] = pd.to_datetime(df_site_kelp['survey_year'].astype(str) + '-01-01')
    
    # Group by site_name and calculate rolling differences
    df_site_kelp['survey_change'] = df_site_kelp.groupby('site_name')['survey_mean'].diff()

    ## select long-term consistent surveys
    # Group the data by site_name and count the number of records for each site
    site_counts = df_site_kelp.groupby('site_name').size()
    # Identify sites with only one record
    single_survey_sites = site_counts[site_counts == 1].index

    df_site_kelp = df_site_kelp[~df_site_kelp['site_name'].isin(single_survey_sites)]
    if input_location != 'Maria Island':
        # Group by site_name and get the number of unique years for each site
        site_years = df_site_kelp.groupby('site_name')['survey_year'].nunique()
        if input_location == 'Jervis Bay':
            consistent_sites = site_years[site_years >= 10].index.tolist()
        else:    
            consistent_sites = site_years[site_years >= 7].index.tolist()
        # Now, filter the main dataframe to keep only these sites
        df_site_kelp = df_site_kelp[df_site_kelp['site_name'].isin(consistent_sites)]
    
    return df_site_kelp

In [ ]:
df_site_kelp = get_site_year(input_location)
df_site_kelp

#### Add summer mhw metrics in the df

In [ ]:
# Initialize a new column in df_site_kelp to store the computed max summer temperatures
df_site_kelp['summer_temp_max'] = np.nan
df_site_kelp['summer_dhd50_max'] = np.nan
df_site_kelp['summer_inten_max'] = np.nan
df_site_kelp['summer_dtdt_max'] = np.nan

for site in df_site_kelp.site_name.unique():
    dfi = df_site_kelp[df_site_kelp.site_name == site]
    
    for j in range(1, len(dfi.survey_year)):             # survey_year in dfi is not continuous, so
        # find the max temp between the two survey years. (j-1)th is not just one-year before jth year , it is the previous survey year before jth survey year 
        summer_temp_max = summer_mhw_metrix.summer_temp[summer_mhw_metrix.survey_year.isin(range(dfi.survey_year.iloc[j-1]+1, dfi.survey_year.iloc[j]+1))].max() 
        summer_dhd50_max = summer_mhw_metrix.summer_dhd50[summer_mhw_metrix.survey_year.isin(range(dfi.survey_year.iloc[j-1]+1, dfi.survey_year.iloc[j]+1))].max()
        summer_inten_max = summer_mhw_metrix.summer_inten[summer_mhw_metrix.survey_year.isin(range(dfi.survey_year.iloc[j-1]+1, dfi.survey_year.iloc[j]+1))].max()
        summer_dtdt_max = summer_mhw_metrix.summer_dtdt[summer_mhw_metrix.survey_year.isin(range(dfi.survey_year.iloc[j-1]+1, dfi.survey_year.iloc[j]+1))].max() 
        
        # Update the summer_temp_max value in the corresponding row of df_site_kelp
        mask = (df_site_kelp.site_name == site) & (df_site_kelp.survey_year == dfi.survey_year.iloc[j])
        df_site_kelp.loc[mask, 'summer_temp_max'] = summer_temp_max
        df_site_kelp.loc[mask, 'summer_dhd50_max'] = summer_dhd50_max
        df_site_kelp.loc[mask, 'summer_inten_max'] = summer_inten_max
        df_site_kelp.loc[mask, 'summer_dtdt_max'] = summer_dtdt_max

# Now df_site_kelp has an additional column 'summer_temp_max' with the computed max summer temperatures
df_site_kelp = df_site_kelp.dropna()
df_site_kelp

In [ ]:
## ======= saving processed data ==========
## Comment lines to change the order based on location!!!
# df_site_kelp.to_csv('4pop_change_rawdata.csv', index=False)
df_existing = pd.read_csv('4pop_change_rawdata.csv')
df_combined = pd.concat([df_existing, df_site_kelp], ignore_index=True) 
df_combined.to_csv('4pop_change_rawdata.csv', index=False)

#### Fitting a model for dhd!

In [ ]:
x = df_site_kelp.summer_dhd50_max
y = df_site_kelp.survey_change

In [ ]:
model = smf.mixedlm("survey_change ~ summer_dhd50_max", df_site_kelp, groups = df_site_kelp["site_name"])
fit = model.fit(method='powell')

# Predict y-values for the regression line using the range of 'summer_temp' values
x_vals = np.linspace(df_site_kelp['summer_dhd50_max'].min(), df_site_kelp['summer_dhd50_max'].max(), 1000)
y_vals = fit.predict(pd.DataFrame({"summer_dhd50_max": x_vals}))

# Get the summary
fit.summary()

In [ ]:
fig = px.scatter(x='summer_dhd50_max', y='survey_change', data_frame = df_site_kelp, color='site_name', opacity=0.6)
fig.update_traces(marker_size=10 ,selector=dict(mode='markers'))

# Add the regression line to the scatter plot
fig.add_traces(go.Scatter(x = x_vals, y = y_vals, mode='lines', name='Linear Mixed Model', line=dict(color='black', dash='dash')))

fig.update_layout(autosize=False, width=800, height=600, 
                  title=f"Linear mixed-effects regression of kelp change in {input_location}", xaxis_title= u'Accumulated intensity (\u00B0C•day)', yaxis_title='Canopy Cover absolute change (%)', 
                  title_font=dict(family="Abadi, Arial, sans-serif", size=20), xaxis_title_font=dict(family="Abadi, Arial, sans-serif", size=18), yaxis_title_font=dict(family="Abadi, Arial, sans-serif", size=18),
                  font=dict(family="Abadi, Arial, sans-serif", size=14))
fig.show()

#### Confidence interval for dhd!

In [ ]:
import warnings
from statsmodels.tools.sm_exceptions import ConvergenceWarning

# Number of bootstrap samples
n_bootstrap = 1000

# Placeholders for our predictions
bootstrap_preds = np.zeros((n_bootstrap, len(x_vals)))

with warnings.catch_warnings():
    warnings.simplefilter("ignore", category=ConvergenceWarning)
    
    for i in range(n_bootstrap):
        # Resample the dataset with replacement
        resampled = df_site_kelp.sample(n=len(df_site_kelp), replace=True)
        
        # Refit the mixed-effects model to the resampled data
        model = smf.mixedlm("survey_change ~ summer_dhd50_max", resampled, groups=resampled["site_name"])
        fit = model.fit(method='powell', maxiter=1000, disp=0)  # increased maxiter
        
        # Predict using the new model
        y_vals_bootstrap = fit.predict(pd.DataFrame({"summer_dhd50_max": x_vals}))
        bootstrap_preds[i, :] = y_vals_bootstrap

# Calculate the 2.5th and 97.5th percentiles for the predictions
lower_bound = np.percentile(bootstrap_preds, 2.5, axis=0)
upper_bound = np.percentile(bootstrap_preds, 97.5, axis=0)

# Now you can plot your predictions and the confidence intervals
plt.plot(x_vals, y_vals, 'b-') 
plt.fill_between(x_vals, lower_bound, upper_bound, color='gray', alpha=0.5) 

In [ ]:
df_model = pd.DataFrame({"location": input_location,
                         "summer_dhd50_max": x_vals,
                         "predicted_change": y_vals,
                         'lower_bound': lower_bound, 
                         'upper_bound': upper_bound
                        })
# df_model.to_csv('dhdmodel_change_relationship.csv', index=False)


In [ ]:
## ======= saving processed data ==========
df_existing = pd.read_csv('dhdmodel_change_relationship.csv')
df_combined = pd.concat([df_existing, df_model], ignore_index=True) 
df_combined.to_csv('dhdmodel_change_relationship.csv', index=False)

#### Fitting a model for summer_temp!

In [ ]:
x = df_site_kelp.summer_temp_max
y = df_site_kelp.survey_change

In [ ]:
model = smf.mixedlm("survey_change ~ summer_temp_max", df_site_kelp, groups = df_site_kelp["site_name"])
fit = model.fit(method='powell')

# Predict y-values for the regression line using the range of 'summer_temp' values
x_vals = np.linspace(df_site_kelp['summer_temp_max'].min(), df_site_kelp['summer_temp_max'].max(), 1000)
y_vals = fit.predict(pd.DataFrame({"summer_temp_max": x_vals}))

# Get the summary
fit.summary()

In [ ]:
fig = px.scatter(x='summer_temp_max', y='survey_change', data_frame = df_site_kelp, color='site_name', opacity=0.6)
fig.update_traces(marker_size=10 ,selector=dict(mode='markers'))

# Add the regression line to the scatter plot
fig.add_traces(go.Scatter(x = x_vals, y = y_vals, mode='lines', name='Linear Mixed Model', line=dict(color='black', dash='dash')))

fig.update_layout(autosize=False, width=800, height=600, 
                  title=f"Linear mixed-effects regression of kelp change in {input_location}", xaxis_title= u'Temperature (\u00B0C)', yaxis_title='Canopy Cover absolute change (%)', 
                  title_font=dict(family="Abadi, Arial, sans-serif", size=20), xaxis_title_font=dict(family="Abadi, Arial, sans-serif", size=18), yaxis_title_font=dict(family="Abadi, Arial, sans-serif", size=18),
                  font=dict(family="Abadi, Arial, sans-serif", size=14))
fig.show()

#### confidence interval for summer_temp!

In [ ]:
n_bootstrap = 1000

# Placeholders for our predictions
bootstrap_preds = np.zeros((n_bootstrap, len(x_vals)))

with warnings.catch_warnings():
    warnings.simplefilter("ignore", category=ConvergenceWarning)
    
    for i in range(n_bootstrap):
        # Resample the dataset with replacement
        resampled = df_site_kelp.sample(n=len(df_site_kelp), replace=True)
        model = smf.mixedlm("survey_change ~ summer_temp_max", resampled, groups=resampled["site_name"])
        fit = model.fit(method='powell', maxiter=1000, disp=0)  # increased maxiter
 
        y_vals_bootstrap = fit.predict(pd.DataFrame({"summer_temp_max": x_vals}))
        bootstrap_preds[i, :] = y_vals_bootstrap

lower_bound = np.percentile(bootstrap_preds, 2.5, axis=0)
upper_bound = np.percentile(bootstrap_preds, 97.5, axis=0)

# Now you can plot your predictions and the confidence intervals
plt.plot(x_vals, y_vals, 'b-')  # blue line for the prediction
plt.fill_between(x_vals, lower_bound, upper_bound, color='gray', alpha=0.5) 

In [ ]:
df_model = pd.DataFrame({"location": input_location,
                         "summer_temp_max": x_vals,
                         "predicted_change": y_vals,
                         'lower_bound': lower_bound, 
                         'upper_bound': upper_bound
                        })
# df_model.to_csv('model_change_relationship.csv', index=False)


In [ ]:
## ======= saving processed data ==========
df_existing = pd.read_csv('model_change_relationship.csv')
df_combined = pd.concat([df_existing, df_model], ignore_index=True) 
df_combined.to_csv('model_change_relationship.csv', index=False)

#### Fitting model for max intensity!

In [ ]:
x = df_site_kelp.summer_inten_max
y = df_site_kelp.summer_inten_max

In [ ]:
model = smf.mixedlm("survey_change ~ summer_inten_max", df_site_kelp, groups = df_site_kelp["site_name"])
fit = model.fit(method='powell')
x_vals = np.linspace(df_site_kelp['summer_inten_max'].min(), df_site_kelp['summer_inten_max'].max(), 1000)
y_vals = fit.predict(pd.DataFrame({"summer_inten_max": x_vals}))

fit.summary()

In [ ]:
fig = px.scatter(x='summer_inten_max', y='survey_change', data_frame = df_site_kelp, color='site_name', opacity=0.6)
fig.update_traces(marker_size=10 ,selector=dict(mode='markers'))

# Add the regression line to the scatter plot
fig.add_traces(go.Scatter(x = x_vals, y = y_vals, mode='lines', name='Linear Mixed Model', line=dict(color='black', dash='dash')))

# fig.add_trace(go.Scatter(x=x_vals, y=y_vals_linear, mode='lines', name="Linear Regression", marker_color="black"))

fig.update_layout(autosize=False, width=800, height=600, 
                  title=f"Linear mixed-effects regression of kelp change in {input_location}", xaxis_title= u'Max intensity (\u00B0C)', yaxis_title='Canopy Cover absolute change (%)', 
                  title_font=dict(family="Abadi, Arial, sans-serif", size=20), xaxis_title_font=dict(family="Abadi, Arial, sans-serif", size=18), yaxis_title_font=dict(family="Abadi, Arial, sans-serif", size=18),
                  font=dict(family="Abadi, Arial, sans-serif", size=14))
fig.show()

#### Confidence interval for intensity!

In [ ]:
n_bootstrap = 1000

# Placeholders for our predictions
bootstrap_preds = np.zeros((n_bootstrap, len(x_vals)))

with warnings.catch_warnings():
    warnings.simplefilter("ignore", category=ConvergenceWarning)
    
    for i in range(n_bootstrap):
        # Resample the dataset with replacement
        resampled = df_site_kelp.sample(n=len(df_site_kelp), replace=True)
        
        # Refit the mixed-effects model to the resampled data
        model = smf.mixedlm("survey_change ~ summer_inten_max", resampled, groups=resampled["site_name"])
        fit = model.fit(method='powell', maxiter=1000, disp=0)  # increased maxiter
        
        # Predict using the new model
        y_vals_bootstrap = fit.predict(pd.DataFrame({"summer_inten_max": x_vals}))
        bootstrap_preds[i, :] = y_vals_bootstrap

# Calculate the 2.5th and 97.5th percentiles for the predictions
lower_bound = np.percentile(bootstrap_preds, 2.5, axis=0)
upper_bound = np.percentile(bootstrap_preds, 97.5, axis=0)

# Now you can plot your predictions and the confidence intervals
plt.plot(x_vals, y_vals, 'b-')  # blue line for the prediction
plt.fill_between(x_vals, lower_bound, upper_bound, color='gray', alpha=0.5)  

In [ ]:
df_model = pd.DataFrame({"location": input_location,
                         "summer_inten_max": x_vals,
                         "predicted_change": y_vals,
                         'lower_bound': lower_bound, 
                         'upper_bound': upper_bound
                        })
# df_model.to_csv('inten_max_model_change_relationship.csv', index=False)


In [ ]:
## ======= saving processed data ==========
df_existing = pd.read_csv('inten_max_model_change_relationship.csv')
df_combined = pd.concat([df_existing, df_model], ignore_index=True) 
df_combined.to_csv('inten_max_model_change_relationship.csv', index=False)

#### Fitting a model for dtdt!

In [ ]:
x = df_site_kelp.summer_dtdt_max
y = df_site_kelp.survey_change

In [ ]:
model = smf.mixedlm("survey_change ~ summer_dtdt_max", df_site_kelp, groups = df_site_kelp["site_name"])
fit = model.fit(method='powell')

x_vals = np.linspace(df_site_kelp['summer_dtdt_max'].min(), df_site_kelp['summer_dtdt_max'].max(), 100)
y_vals = fit.predict(pd.DataFrame({"summer_dtdt_max": x_vals}))
fit.summary()

In [ ]:
fig = px.scatter(x='summer_dtdt_max', y='survey_change', data_frame = df_site_kelp, color='site_name', opacity=0.6)
fig.update_traces(marker_size=10 ,selector=dict(mode='markers'))

fig.add_traces(go.Scatter(x = x_vals, y = y_vals, mode='lines', name='Linear Mixed Model', line=dict(color='black', dash='dash')))

fig.update_layout(autosize=False, width=800, height=600, 
                  title = f"Linear mixed-effects regression of kelp change in {input_location}", xaxis_title= u'Max Temperature tendency (\u00B0C/day)', yaxis_title='Canopy Cover absolute change (%)', 
                  title_font=dict(family="Abadi, Arial, sans-serif", size=20), xaxis_title_font=dict(family="Abadi, Arial, sans-serif", size=18), yaxis_title_font=dict(family="Abadi, Arial, sans-serif", size=18),
                  font=dict(family="Abadi, Arial, sans-serif", size=14))
fig.show()

#### Confidence interval for dtdt!

In [ ]:
n_bootstrap = 1000
bootstrap_preds = np.zeros((n_bootstrap, len(x_vals)))

with warnings.catch_warnings():
    warnings.simplefilter("ignore", category=ConvergenceWarning)
    
    for i in range(n_bootstrap):
        # Resample the dataset with replacement
        resampled = df_site_kelp.sample(n=len(df_site_kelp), replace=True)
        
        # Refit the mixed-effects model to the resampled data
        model = smf.mixedlm("survey_change ~ summer_dtdt_max", resampled, groups=resampled["site_name"])
        fit = model.fit(method='powell', maxiter=1000, disp=0)  # increased maxiter
        
        # Predict using the new model
        y_vals_bootstrap = fit.predict(pd.DataFrame({"summer_dtdt_max": x_vals}))
        bootstrap_preds[i, :] = y_vals_bootstrap

lower_bound = np.percentile(bootstrap_preds, 2.5, axis=0)
upper_bound = np.percentile(bootstrap_preds, 97.5, axis=0)

# Now you can plot your predictions and the confidence intervals
plt.plot(x_vals, y_vals, 'b-')  # blue line for the prediction
plt.fill_between(x_vals, lower_bound, upper_bound, color='gray', alpha=0.5)  

In [ ]:
df_model = pd.DataFrame({"location": input_location,
                         "summer_dtdt_max": x_vals,
                         "predicted_change": y_vals,
                         'lower_bound': lower_bound, 
                         'upper_bound': upper_bound
                        })
## ======= saving processed data ==========
# df_model.to_csv('dtdtmodel_change_relationship.csv', index=False)

In [ ]:
## ======= saving processed data ==========
df_existing = pd.read_csv('dtdtmodel_change_relationship.csv')
df_combined = pd.concat([df_existing, df_model], ignore_index=True) 
df_combined.to_csv('dtdtmodel_change_relationship.csv', index=False)